In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score
%matplotlib inline
import keras

# Load Test Dataset

In [2]:
df_test = pd.read_csv("All_feature_Independent_dataframe.csv")

# NetSurfP-2.0 Features

In [3]:
X_test_NetSurfP2 = df_test.iloc[:,2100:2428]

y_test_NetSurfP2 = df_test["Target"]


print(X_test_NetSurfP2.isnull().sum().sum())
X_test_NetSurfP2.fillna(0,inplace=True)
print(X_test_NetSurfP2.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_NetSurfP2 = scaler.fit_transform(X_test_NetSurfP2)


X_test_NetSurfP2 = np.array(X_test_NetSurfP2)
y_test_NetSurfP2 = np.array(y_test_NetSurfP2)

1680
0


# PSSM Features

In [4]:
X_test_PSSM_test = df_test.iloc[:,1280:2100]

y_test_PSSM = df_test["Target"]

print(X_test_PSSM_test.isnull().sum().sum())
X_test_PSSM_test.fillna(0,inplace=True)
print(X_test_PSSM_test.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_PSSM_test = scaler.fit_transform(X_test_PSSM_test)


X_test_PSSM_test = np.array(X_test_PSSM_test)
y_test_PSSM = np.array(y_test_PSSM)

4200
0


# Gapped Dipeptide Features

In [5]:
X_test_Gapped_Dipeptide = df_test.iloc[:,2428:2468]

y_test_Gapped_Dipeptide = df_test["Target"]


scaler = MinMaxScaler()


X_test_Gapped_Dipeptide = scaler.fit_transform(X_test_Gapped_Dipeptide)


X_test_Gapped_Dipeptide = np.array(X_test_Gapped_Dipeptide)
y_test_Gapped_Dipeptide = np.array(y_test_Gapped_Dipeptide)

# Combine Three features and check the shape of the features

In [7]:
X_test_PSSM_NetSurfP2_Gapped_Dipeptide = np.hstack((X_test_PSSM_test,X_test_NetSurfP2,X_test_Gapped_Dipeptide))
y_test_PSSM_NetSurfP2_Gapped_Dipeptide = y_test_Gapped_Dipeptide

X_test = X_test_PSSM_NetSurfP2_Gapped_Dipeptide
y_test = y_test_PSSM_NetSurfP2_Gapped_Dipeptide

X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_test.shape)
print(y_test.shape)

(447, 1188)
(447,)


# Load the model and see the performance matrices

In [18]:
model = keras.models.load_model("Concomitant13.h5")
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.5)
y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
y_pred = np.array(y_pred)
print("Matthews Correlation : ",matthews_corrcoef(y_test, y_pred))
print("Confusion Matrix : \n",confusion_matrix(y_test, y_pred))
print("Accuracy on test set:   ",accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
print(mcc)

TP = cm[1][1]
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]

Sensitivity = TP/(TP+FN)

Specificity = TN/(TN+FP)

print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

print(classification_report(y_test, y_pred))

fpr, tpr, _ = roc_curve(y_test, y_pred)

roc_auc_test = auc(fpr,tpr)

print("Area Under Curve:   ",roc_auc_test)

Matthews Correlation :  0.5646102635159639
Confusion Matrix : 
 [[210  70]
 [ 28 139]]
Accuracy on test set:    0.7807606263982103
0.5646102635159639
Sensitivity:    0.8323353293413174 	 Specificity:    0.75
              precision    recall  f1-score   support

           0       0.88      0.75      0.81       280
           1       0.67      0.83      0.74       167

    accuracy                           0.78       447
   macro avg       0.77      0.79      0.78       447
weighted avg       0.80      0.78      0.78       447

Area Under Curve:    0.7911676646706587
